In [88]:
import pandas as pd
import requests
import warnings
import time
import multiprocessing
import threading
from concurrent.futures import ThreadPoolExecutor
from ipapy import is_valid_ipa
# from google.colab import files
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

In [170]:
df = pd.read_csv("ebanglalibrary_100k_with_ipa0_106500_validated_0_106500.csv")
df_words=pd.read_csv("ipa_validation_batch1.xlsx - Sheet1.csv")


In [157]:
def replace_func(subset):
  for row in subset.itertuples():
    ipa=row.ipa+"%"
    for i, ch in enumerate(ipa):
      if ch=='ʲ' and is_valid_ipa(ipa[i+1])==False:
        ipa = ipa[:i] + "e̯" + ipa[i+1:]
    ipa=ipa.replace("a","ɐ")
    ipa=ipa.replace("i","ɪ")
    ipa=ipa.replace("æ","ɛ")
    ipa=ipa.replace("r","ɾ")
    ipa=ipa[:-1]
    df.at[row.Index,"validated_ipa"]=ipa

    

    
    # words=row.sentence.split(" ")
    # for word in words:
    #   if word[-1]=='ʲ':
    #     word=word[:-1] + "e̯"
        
    #   for ch in word:
        

In [93]:
start=0
steps=10000
end=106500
subsets=[]
start_pos=start
while start<end:
  subsets.append(df.iloc[start:start+steps])
  start=start+steps
pool=ThreadPoolExecutor(len(subsets))
ws=[]
for subset in subsets:
  ws.append(pool.submit(replace_func, subset))
print(pool.shutdown())

None


In [94]:
df.to_csv(f"ebanglalibrary_100k_with_ipa0_106500_validated_{start_pos}_{end}.csv",index=False)

In [158]:
prefix_sentence="__$$__"
prefix_words="__##__"

In [171]:
def replace_words(subset):
  all_sentences=""
  for row in subset.itertuples():
    if all_sentences=="":
      all_sentences=row.sentence
    else:
      all_sentences=all_sentences+prefix_sentence+row.sentence
  for row in df_words.itertuples():
    if pd.isna(row.ɪpas) or pd.isna(row.words):
      continue
    all_sentences=all_sentences.replace(" "+row.words+" "," "+prefix_words+row.ɪpas+" ")
  all_sentences_arr=all_sentences.split(prefix_sentence);
  for row,sentence_v in zip(subset.itertuples(),all_sentences_arr):
    
    validated_ipa=row.validated_ipa.split(" ")
    new_ipa=sentence_v.split(" ")
    # print(validated_ipa)
    # print(new_ipa)
    # print(validated_ipa[0])
    # print("++++++++++++++++++++++++++++++++++++++")
    if new_ipa[0]=="":
      new_ipa=new_ipa[1:]
    if validated_ipa[0]=="":
      validated_ipa=validated_ipa[1:]

    for i,valid in enumerate( new_ipa):
      if valid.startswith(prefix_words):
        validated_ipa[i]=valid
    validated_ipa=" ".join(validated_ipa)
    validated_ipa=validated_ipa
    # .replace(prefix_words,"")
    print(validated_ipa)
    print(row.ipa)
    print(row.sentence)
    print("++++++++++++++++++++++++++++++++++++++")
    df.at[row.Index,"validated_ipa"]=validated_ipa
    df.at[row.Index,"done"]="OK"
      
    # df.at[row.Index,"temp_sentence"]=all_sentences_arr[row.Index+1]

    

In [172]:
# threadpool make sure everything is currect before running this cell
start=0
steps=10
end=100
subsets=[]
start_pos=start
while start<end:
  subsets.append(df.iloc[start:start+steps])
  start=start+steps
pool=ThreadPoolExecutor(len(subsets))
ws=[]
for subset in subsets:
  ws.append(pool.submit(replace_words, subset))
print(pool.shutdown())


9nobbobʱɐɾɔt̪o (ʃoɽʃ kʰɔndo, bo͡ɪ̯ʃɐkʰ-co͡ɪ̯t̪ɾo, 1305), 280 __##__d̪oʃ __##__ɾobɪnd̪ɾonɐt̪ʰ tʰɐkuɾ, __##__pɾoʃɔŋgo kɔt̪ʰɐ।ʃɐkkogɔn d̪ebpuɟok cʰɪlen; ʃo͡ɪ̯bo __##__cʰɪlen bolɪʲɐ͡ɪ̯ __##__pɾomɐn __##__pɾɐpt̪o __##__hoʷɐ ɟɐe̯।
 ʃakkogɔn d̪ebpuɟok cʰilen; ʃo͡i̯bo cʰilen boliʲa͡i̯ proman prapt̪o hoʷa ɟaʲ।
শাক্যগণ দেবপূজক ছিলেন; শৈব ছিলেন বলিয়াই প্রমাণ প্রাপ্ত হওয়া যায়।
++++++++++++++++++++++++++++++++++++++
kɐlokɾome __##__ɐɾɔo̯ __##__nɐnɐ pɾɐcco __##__ɟɔnopɔd̪e ɐɾɟo pɾot̪ɐp bɛpt̪o ho͡ɪ̯ʲɐ poɽɪʲɐcʰɪlo।
 kalokrome arɔ͡o̯ nana pracco ɟɔnopɔd̪e arɟo prot̪ap bæpt̪o ho͡i̯ʲa poɽiʲacʰilo।
কালক্রমে আরও নানা প্রাচ্য জনপদে আৰ্য প্ৰতাপ ব্যাপ্ত হইয়া পড়িয়াছিল।
++++++++++++++++++++++++++++++++++++++
ɪhɐ __##__ʃɔt̪t̪o ho͡ɪ̯le, nɪɾɔkkʰoɾ __##__lokeɾ __##__ɐɾ __##__ɔpoɾɐd̪ʱ kɪ?
 iha ʃot̪t̪o ho͡i̯le, nirɔkkʰor loker ar ɔporad̪ʱ ki?
ইহা সত্য হইলে, নিরক্ষর লোকের আর অপরাধ কী?
++++++++++++++++++++++++++++++++++++++

 9nobbobʱarɔt̪o (ʃoɽʃ kʰɔndo, bo͡i̯ʃakʰ-co͡i̯t̪ro, 1305), 280 10 robind̪ronat̪ʰ tʰakur, pr

In [108]:
for row in df.itertuples():
  if pd.isna(row.done):
    print(f"{row.Index} {row.sentence}")
  if row.Index>10001:
    break

1 তবে ১৮৭২ খ্রিস্টাব্দে বঙ্গদর্শন-এর প্রকাশ এই ধারায় গতিবেগের সঞ্চার ঘটায়।
2 বস্তুত বাগ্মিতায় অক্ষয়কুমারের ছিল জন্মগত অধিকার।
3 ভারতীয় শিল্পের কথা নামক পুস্তকে ইতিপূর্বে শিল্প সম্বন্ধীয় তাঁর নিবন্ধগুলি প্রকাশিত।
4 অন্যান্য বিষয়ের সঙ্গে এ পত্রিকায় ঐতিহাসিক প্রবন্ধাবলিও প্রকাশিত হতে থাকে।
5 সেই অতীত ইতিহাস যেকোনো পাশ্চাত্য সভ্যতা অপেক্ষা খর্ব নয়, এই প্রথম অনুধাবন করল ভারতবাসী।
6 পরে সাধনা-র প্রকাশ বন্ধ হয়ে গেলে সিরাজদ্দৌল্লা-র অবশিষ্টাংশ ভারতী-তে প্রকাশিত হয়।
7 সে জন্য প্রাচীন বাংলার ইতিহাস রচনায় শিলালেখ বা তাম্রলেখ তাঁর কাছে খুব গুরুত্বপূর্ণ উপাদান হিসেবে স্বীকৃতি পেয়েছিল।
8 সেই জন্য বাঙ্গালার ইতিহাস আৰ্য্যাবর্তের ইতিহাসের অংশমাত্র–তাহা হইতে স্বতন্ত্র বা পৃথক নহে।এখন সমগ্র আর্যাবর্তের অধিবাসী অধঃপতিত বলিয়া পরিচিত।
9 বাঙ্গালীমাত্রে ইহার সহায়তাসাধন না করিলে, এই কঠিন ব্রত প্রতিপালিত হইতে পারে না।
10 কালক্রমে আরও নানা প্রাচ্য জনপদে আৰ্য প্ৰতাপ ব্যাপ্ত হইয়া পড়িয়াছিল।
11 তাহার প্রভাবে বাঙ্গালী মনুষ্যত্ব হারাইয়া, আত্মোন্নতিলাভের অযোগ্য হইয়া পড়িতেছিল।
12 আর্যাবর্তের সংস্কৃত

In [107]:
df.to_csv(f"ebanglalibrary_100k_with_ipa0_106500_validated_{start_pos}_{end}.csv",index=False)

In [23]:
name="sourav ahmed"
# name[-1]="x"
name=name[:-1] + "x"
print(name.replace("a","A"))
print(name)

sourAv Ahmex
sourav ahmex


In [109]:
unique_char_bangla=[' ', '!', '#', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '~', '\xa0', '£', 'ª', '\xad', '¯', '·', 'Æ', 'â', '˃', 'ا', 'ث', 'خ', 'د', 'ر', 'س', 'ص', 'ع', 'ق', 'ل', 'ه', 'ي', '।', '॥', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ৗ', 'ড়', 'ঢ়', 'য়', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯', 'ৰ', 'ৱ', '৷', '৺', 'ਆ', '\u200c', '\u200d', '‒', '–', '—', '‘', '’', '“', '”', '•', '…', '−', '√', '∣', '⋯', '\uf020', '\uf02c', '\uf02d', '\uf05b', '\ufeff']
unique_char_ipa=[' ', '!', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '~', '\xa0', '£', '\xad', 'æ', 'ŋ', 'ɔ', 'ɟ', 'ɽ', 'ʃ', 'ʰ', 'ʱ', 'ʲ', 'ʷ', '̃', '̪', '̯', '͡', 'ا', 'ث', 'خ', 'د', 'ر', 'س', 'ص', 'ع', 'ق', 'ل', 'ه', 'ي', '।', '॥', '৷', '–', '—', '‘', '’', '“', '”', '•', '…', '∣', '\ufeff']

# for row in df.itertuples():
#   for ch in row.ipa:
#     if ch not in unique_char_ipa:
#       unique_char_ipa.append(ch)
# unique_char_ipa.sort()
# print(unique_char_ipa)

In [163]:
print(" ".join(unique_char_ipa))

  ! % ' ( ) * , - . / 0 1 2 3 4 5 6 7 8 9 : ; < = > ? A B C D E F G H I J K L M N O P Q R S T U V W Y [ ] _ ` a b c d e f g h i j k l m n o p q r s t u v w x y z { | ~   £ ­ æ ŋ ɔ ɟ ɽ ʃ ʰ ʱ ʲ ʷ ̃ ̪ ̯ ͡ ا ث خ د ر س ص ع ق ل ه ي । ॥ ৷ – — ‘ ’ “ ” • … ∣ ﻿


MISC


In [ ]:
      # if ch=='ʲ' and (row.sentence[i+1]==" " or row.sentence[i+1]=="\n" or  row.sentence[i+1]=="।" or row.sentence[i+1]=="?" or row.sentence[i+1]=="!" or row.sentence[i+1]=="%" or ):


In [111]:
# for item1, item2, item3 in zip([1, 2, 3,], ['a', 'b', 'c','d'], [True, False, True,'d']):
#     print(item1, item2, item3)
    

1 a True
2 b False
3 c True
